In [5]:
class World:
    def __init__(self, n_farmers, n_bandits, n_soilders):

        #initial setup
        self.people = []
        self.farmer_pop = n_farmers
        self.bandit_pop = n_bandits
        self.soiler_pop = n_bandits

        for i in range(n_farmers):
            self.people.append(Person('farmer'))
        for i in range(n_bandits):
            self.people.append(Person('bandit'))
        for i in range(n_soilders):
            self.people.append(Person('soldier'))
        
        self.update_population_counts()

        # global violence metrics
        self.total_robberies = 0
        self.total_suppressions = 0

    # Utility: count role populations
    def update_population_counts(self):
        self.farmer_pop = sum(p.is_farmer() for p in self.people)
        self.bandit_pop = sum(p.is_bandit() for p in self.people)
        self.soldier_pop = sum(p.is_soldier() for p in self.people)

    # Main tick update function, will run once per iteration
    def step(self):
        for person in self.people:
            person.reset_for_new_tick()

        # Production: farmers produce wealth
        for person in self.people:
            if person.is_farmer():
                person.produce(farmer_productivity=5.0)

        # Robberies: bandits attempt to steal from farmers
        farmers = [p for p in self.people if p.is_farmer()]
        bandits = [p for p in self.people if p.is_bandit()] # 
        random.shuffle(bandits)

        self.total_robberies = 0

        for bandit in bandits:
            if not farmers:
                break
            target = random.choice(farmers)
            stole_amount = bandit.attempt_rob(target, steal_rate=2.0)
            if stole_amount[0] > 0:
                self.total_robberies += 1

        # Suppressions: Soldiers suppress bandits
        soldiers = [p for p in self.people if p.is_soldier()]
        bandits = [p for p in self.people if p.is_bandit()]

        self.total_suppressions = 0

        for soldier in soldiers:
            if not bandits:
                break
            target = random.choice(bandits)
            hit = soldier.attempt_suppress(target, p_hit = 0.8, confiscate_rate=1.5)
            if hit:
                self.total_suppressions += 1

        # (4) Tax Farmers and Pay Soldiers
        tax_pool = 0.0
        for farmer in farmers:
            tax_pool += farmer.pay_tax(tax_rate=0.10)

        if soldiers:
            pay_per_soldier = tax_pool / len(soldiers)
        else:
            pay_per_soldier = 0.0

        for soldier in soldiers:
            soldier.receive_pay(pay_per_soldier)

        # Calculating utilities
        self.expected_prod_per_farmer = (sum(p.last_production for p in farmers) / len(farmers) if farmers else 0.0)
        self.expected_rob_per_bandit = (sum(p.last_rob_gain for p in bandits) / len(bandits) if bandits else 0.0)
        self.expected_soldier_pay = (sum(p.last_pay for p in soldiers) / len(soldiers) if soldiers else 0.0)

        self.utilities_for_roles = {
            "farmer": self.expected_prod_per_farmer,
            "bandit": self.expected_rob_per_bandit,
            "soldier": self.expected_soldier_pay
        }

        # Psychological updates
        for person in self.people:
            person.update_psychology(global_violence=self.total_robberies + self.total_suppressions)

        # Role-switch decisions
        for person in self.people:
            person.decide_switch(loyalty_value=person.loyalty, utilities=self.utilities_for_roles, switch_urge_threshold=0.3, utility_margin=0.2)

        # Finalize switching
        for person in self.people:
            person.finalize_role_switch()

        # update population counts
        self.update_population_counts()

    # Export world state (for logging or data collection)
    def to_dict(self):
        return {
            "farmer_pop": self.farmer_pop,
            "bandit_pop": self.bandit_pop,
            "soldier_pop": self.soldier_pop,
            "total_robberies": self.total_robberies,
            "total_suppressions": self.total_suppressions,
            "people": [p.to_dict() for p in self.people]
        }